In [1]:
import tempfile

import pandas as pd
import csv
import numpy as np
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
train = pd.read_csv('tmp/train.csv',index_col = 0)
test = pd.read_csv('tmp/test.csv',index_col = 0)

/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
train_set = train
split_ratio = 0.8

train_set = train_set.sample(frac=1, random_state=6)
val_set = train_set[int(split_ratio*train_set.shape[0]):]
train_set = train_set[:int(split_ratio*train_set.shape[0])]

y_train = train_set['target']
train_set.drop('target', axis=1, inplace=True)

y_val = val_set['target']
val_set.drop('target', axis=1, inplace=True)

In [4]:
#Designate the target feature name and the features to be used in the dataset

FEATURES = list(test.columns)
LABEL = 'target'

wide_columns =  [tf.feature_column.numeric_column(key= 'x',
                                       shape=(60*2**5,),
                                       default_value=0,
                                       dtype=tf.int32)]
cross_columns = []
deep_columns = []


In [5]:
def build_estimator(model_dir, model_type):
    if model_type == 'wide':
        model = tf.estimator.LinearClassifier(model_dir=model_dir,
                                              feature_columns=wide_columns + cross_columns)

    elif model_type == 'deep':
        model = tf.estimator.DNNClassifier(model_dir=model_dir,
                                           feature_columns=deep_columns,
                                           hidden_units=[1024, 512, 256],
                                           optimizer=tf.train.AdamOptimizer(learning_rate=0.001,
                                                                            name='Adam'))

    elif model_type == 'combined':
        model = tf.estimator.DNNLinearCombinedClassifier(model_dir=model_dir,
                                                         linear_feature_columns=cross_columns,
                                                         dnn_feature_columns=deep_columns,
                                                         dnn_hidden_units=[100, 50])

    return model


In [6]:
from sklearn.externals import joblib
feature_trans = joblib.load('./tmp/lgb.pkl')
tr_features = feature_trans.predict(train_set.values,pred_leaf=True)
va_features = feature_trans.predict(val_set.values,pred_leaf=True)
tes_features = feature_trans.predict(test.values,pred_leaf=True)

In [21]:
from tensorflow.contrib.data import Dataset
from sklearn.externals import joblib
feature_trans = joblib.load('./tmp/lgb.pkl')


def make_generator(X, Y):

    def _generator():
        for x, y in zip(X, Y):
            yield x, y

    return _generator
def train_input_fn():
  
    dataset = tf.data.Dataset.from_generator(
          make_generator(tr_features, y_train.values),
          (tf.uint8, tf.float32))
    dataset.repeat(None)
    dataset = dataset.map(
        lambda x, y: tuple((tf.reshape(tf.one_hot(x,depth=2**5),(1920,)),y))
    )
    batched_dataset = dataset.batch(100)
    iterator = batched_dataset.make_one_shot_iterator()
    
    x, y = list(iterator.get_next())
    #print(x,y)
    return {'x': x}, tf.expand_dims(y,axis=1)
def valid_input_fn():
  
    dataset = tf.data.Dataset.from_generator(
          make_generator(va_features, y_val.values),
          (tf.uint8, tf.float32))
    dataset.repeat(None)
    dataset = dataset.map(
        lambda x, y: tuple((tf.reshape(tf.one_hot(x,depth=2**5),(1920,)),y))
    )
    batched_dataset = dataset.batch(100)
    iterator = batched_dataset.make_one_shot_iterator()
    
    x, y = list(iterator.get_next())
    #print(x,y)
    return {'x': x}, tf.expand_dims(y,axis=1)
def predict_generator(X):

    def _generator():
        for x in X:
            yield x

    return _generator
def predict_input_fn():
      
    dataset = tf.data.Dataset.from_generator(
          predict_generator(tes_features),
          (tf.uint8))
    dataset.repeat(1)
    """dataset = dataset.map(
    lambda x, y: tuple(tf.py_func(
        _read_py_function, [x, y], [tf.float32, tf.float32])))"""
    dataset = dataset.map(
        lambda x: tf.reshape(tf.one_hot(x,depth=2**5),(1920,))
    )
    batched_dataset = dataset.batch(50000)
    iterator = batched_dataset.make_one_shot_iterator()
    
    x = iterator.get_next()
    #print(x,y)
    return {'x': x}

In [22]:
def input_fn(X, y, mode, batch_size):
    print(X.shape)
    X.fillna(value='unknown', axis=1, inplace=True)    

    if mode == 'train':
        return train_input_fn
    
    elif mode == 'eval':
        return valid_input_fn 
        """pandas_input_fn(x = pd.DataFrame({k: X[k].values for k in FEATURES}),
                                                   y = pd.Series(y.values),
                                                   batch_size=batch_size,
                                                   num_epochs=1,
                                                   shuffle=False,
                                                   num_threads=1,
                                                   target_column='target')"""
    
    elif mode == 'predict':
        return predict_input_fn
    

In [23]:
def train_model(model_dir, model_type, train_steps, X_train, y_train, X_test, y_test, batch_size):

#Create a temporary directory to store the model if no model directory argument is given

    model_dir = tempfile.mkdtemp() if not model_dir else model_dir
    
    print('build_estimator')
    model = build_estimator(model_dir, model_type)
    
    print('train start')
    model.train(input_fn=input_fn(X_train, y_train, mode='train', batch_size=batch_size),
                max_steps=train_steps)
    
#Evaluate the trained model on a separate validation set in n/batch_size steps
    print('eval start')
    model.evaluate(input_fn=input_fn(X_test, y_test, mode='eval', batch_size=batch_size),
                        steps=100)

    print('end!')
    
    return model
    

In [24]:
tf.logging.set_verbosity(tf.logging.INFO)
wide_model = train_model(model_dir='model/', model_type='wide', train_steps=1000,
                         X_train=train_set, y_train=y_train,
                         X_test=val_set, y_test=y_val,
                         batch_size=100)


build_estimator
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6ae854edd8>, '_task_id': 0, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_model_dir': 'model/', '_is_chief': True, '_session_config': None, '_service': None, '_task_type': 'worker', '_master': '', '_num_worker_replicas': 1, '_save_checkpoints_steps': None, '_num_ps_replicas': 0}
train start
(5901934, 16)
INFO:tensorflow:Skipping training since max_steps has already saved.
eval start
(1475484, 16)
INFO:tensorflow:Starting evaluation at 2017-11-25-23:52:37
INFO:tensorflow:Restoring parameters from model/model.ckpt-1000
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/10

"results = wide_model.evaluate(input_fn=input_fn(train_set, y_train, mode='eval', batch_size=1000),\n                              steps=1476)\n"

In [26]:
tf.logging.set_verbosity(tf.logging.INFO)
predictions = wide_model.predict(input_fn=input_fn(test, None, mode='predict',
                                                   batch_size=10000))
count = 0
submission = list()

for row in predictions:
    submission.append(row['probabilities'][1])
    count += 1
    if(count % 1000 == 0):
        print(count)
   


(2556790, 16)
INFO:tensorflow:Restoring parameters from model/model.ckpt-1000
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
1

NameError: name 'ids' is not defined

In [27]:
len(submission)

2556790

In [31]:
ids = pd.read_csv("input/test.csv")['id'].values
result_df = pd.DataFrame()

result_df['id'] = ids
result_df['target'] = np.array(submission)
result_df.to_csv('wide.csv.gz', compression = 'gzip', index=False, float_format = '%.5f')